In [3]:
import sys
import os
script_dir = os.getcwd() # coleata o diretorio do projeto atual
sys.path.append(script_dir)

In [4]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models

In [32]:
def load_data_base(local_base):

    data = []
    subdirectory_classes = [(directory.path, directory.name) for directory in os.scandir(local_base) if directory.is_dir()]
    for x in subdirectory_classes:  
        print(x)
    print(len(subdirectory_classes))

    for directory, label in subdirectory_classes:
        class_images = []
        files = os.listdir(directory)    

        for file in files:
            image_path = os.path.join(directory, file)
            image = cv2.imread(image_path)

            if image is not None:
                class_images.append((image, label))
            else:
                print(f"Erro ao carregar a imagem: {image_path}")
                
        data.append(class_images)

    return data

In [33]:
def contar_imagens_por_pasta(list_class):
    dados = []

    for class_type in list_class:
        classe = class_type[0][1]
       
        dados.append({'Classe': classe, 'Número de Imagens': len(class_type)})

    # Cria um DataFrame com os dados
    df = pd.DataFrame(dados)
    # Ordena o DataFrame pela classe
    df = df.sort_values(by='Classe').reset_index(drop=True)
    
    return df

In [34]:
def split_train_test(data, test_size=0.2):
    train_data = []
    test_data = []

    for class_images in data:
        # Separar as imagens e suas etiquetas
        images, labels = zip(*class_images)
        # Dividir os dados em conjuntos de treinamento e teste
        train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=test_size, random_state=42)
        # Adicionar os conjuntos de treinamento e teste às respectivas listas
        train_data.append(list(zip(train_images, train_labels)))
        test_data.append(list(zip(test_images, test_labels)))

    return train_data, test_data

In [35]:
dir_base_img = script_dir + r'\data'
data_base = load_data_base(dir_base_img)

('c:\\Users\\Pichau\\Desktop\\Faculdade\\Projects\\Facul Repositorios\\Repositorios_Universitarios\\Processamento_imagens\\aula04\\data\\airplane', 'airplane')
('c:\\Users\\Pichau\\Desktop\\Faculdade\\Projects\\Facul Repositorios\\Repositorios_Universitarios\\Processamento_imagens\\aula04\\data\\car', 'car')
('c:\\Users\\Pichau\\Desktop\\Faculdade\\Projects\\Facul Repositorios\\Repositorios_Universitarios\\Processamento_imagens\\aula04\\data\\cat', 'cat')
('c:\\Users\\Pichau\\Desktop\\Faculdade\\Projects\\Facul Repositorios\\Repositorios_Universitarios\\Processamento_imagens\\aula04\\data\\dog', 'dog')
('c:\\Users\\Pichau\\Desktop\\Faculdade\\Projects\\Facul Repositorios\\Repositorios_Universitarios\\Processamento_imagens\\aula04\\data\\flower', 'flower')
('c:\\Users\\Pichau\\Desktop\\Faculdade\\Projects\\Facul Repositorios\\Repositorios_Universitarios\\Processamento_imagens\\aula04\\data\\fruit', 'fruit')
('c:\\Users\\Pichau\\Desktop\\Faculdade\\Projects\\Facul Repositorios\\Repositor

In [36]:
print(len(data_base[0]))

727


In [37]:
train_data, test_data = split_train_test(data_base, 0.2)
contar_imagens_por_pasta(train_data)

,Classe,Número de Imagens
0,airplane,581
1,car,774
2,cat,708
3,dog,561
4,flower,674
5,fruit,800
6,motorbike,630
7,person,788


In [38]:
contar_imagens_por_pasta(test_data)

,Classe,Número de Imagens
0,airplane,146
1,car,194
2,cat,177
3,dog,141
4,flower,169
5,fruit,200
6,motorbike,158
7,person,198


In [39]:
def extract_histogram(image):
    # Converter a imagem para escala de cinza
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Calcular o histograma da imagem
    hist = cv2.calcHist([gray], [0], None, [256], [0,256])
    # Normalizar o histograma
    hist /= hist.sum()
    return hist.flatten()

def train_model(train_data, test_data):
    # Separar características (histogramas) e etiquetas de treinamento
    train_features, train_labels = zip(*train_data)
    train_features = np.array(train_features)
    
    # Separar características (histogramas) e etiquetas de teste
    test_features, test_labels = zip(*test_data)
    test_features = np.array(test_features)

    # Codificar as etiquetas em one-hot vectors
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(train_labels)
    train_labels_encoded = tf.keras.utils.to_categorical(integer_encoded)

    # Construir a arquitetura do modelo
    model = models.Sequential([
        layers.Dense(128, activation='relu', input_shape=(256,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(set(train_labels)), activation='softmax')
    ])

    # Compilar o modelo
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Treinar o modelo
    model.fit(train_features, train_labels_encoded, epochs=10, batch_size=32, verbose=1)

    # Avaliar o modelo no conjunto de teste
    test_loss, test_acc = model.evaluate(test_features, test_labels, verbose=2)
    print('\nTest accuracy:', test_acc)


In [5]:
def create_model(input_dim, output_dim, dense_layers):
    
    model = models.Sequential()
    model.add(layers.Dense(dense_layers[0], activation='relu', input_shape=(input_dim,)))

    for neurons in dense_layers[1:]:
        model.add(layers.Dense(neurons, activation='relu'))

    model.add(layers.Dense(output_dim, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def train_model(train_data, test_data):
    
    # Separar características (histogramas) e etiquetas de treinamento
    train_features, train_labels = zip(*train_data)
    train_features = np.array(train_features)
    
    # Separar características (histogramas) e etiquetas de teste
    test_features, test_labels = zip(*test_data)
    test_features = np.array(test_features)

    # Codificar as etiquetas em one-hot vectors
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(train_labels)
    train_labels_encoded = tf.keras.utils.to_categorical(integer_encoded)

In [6]:
# Exemplo de uso
input_dim = 256
output_dim = 10  # Suponha que temos 10 classes de saída
dense_layers = [128, 10]  # Dois layers densos com 128 e 64 neurônios, respectivamente

model = create_model(input_dim, output_dim, dense_layers)

c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,296 (133.97 KB)

 Trainable params: 34,296 (133.97 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
train_model(train_data, test_data)

ValueError: too many values to unpack (expected 2)